In [51]:
import pandas as pd
import ast
from fastapi import FastAPI
import json
import fastparquet

#rows=[]
#with open('steam_games.json', encoding='utf-8-sig') as f:
 #   for line in f.readlines():
  #    rows.append(ast.literal_eval(line))

#df=pd.DataFrame(rows)

app = FastAPI()

In [77]:
# Lee el archivo Parquet y crea un objeto DataFrame
df = fastparquet.ParquetFile('steam_games.parquet').to_pandas()

# Función para obtener los géneros más vendidos
@app.get("/genero/")
def genero(anio: int):
    df_filtrado = df[df['release_date'] == anio]
    if df_filtrado.empty:
        return {"!": f"No hay datos disponibles para el año {anio}"}
    
    generos_count = {}
    for lista_generos in df_filtrado['genres']:
        if lista_generos is not None:
            for genero_juego in lista_generos:
                if genero_juego is not None:
                    generos_count[genero_juego] = generos_count.get(genero_juego, 0) + 1
    
    generos_top = dict(sorted(generos_count.items(), key=lambda item: item[1], reverse=True)[:5])
    return generos_top


In [93]:
genero(2019)

{'Action': 2, 'Adventure': 2, 'Indie': 2, 'RPG': 2, 'Strategy': 1}

In [95]:
# Función para obtener los juegos lanzados en un año
@app.get("/juegos/")
def juegos(anio: int):
   # Filtrar el DataFrame por el año deseado
    df_filtrado = df[df['release_date'] == anio]
    if df_filtrado.empty:
        return {"!": f"No hay datos disponibles para el año {anio}"}
    
    return df_filtrado["title"].unique()

In [98]:
df_filtrado = df[df['release_date'] == 2015]

In [99]:
df_filtrado['title']

1550                                Assault Android Cactus
1559                       America's Army: Proving Grounds
2488                                             The Tower
2999           Dimmdrive :: Gaming Ramdrive @ 10,000+ MB/s
3001                                          Fake Colours
                               ...                        
27701    Rocksmith® 2014 – Linkin Park - “One Step Closer”
28418                                         Stained Demo
28492                                              Sentris
28630                                         Pixel Piracy
29051                                  RUNNING WITH RIFLES
Name: title, Length: 4945, dtype: object

In [96]:
juegos(2013)

array(['RIFT', 'Trine 2: Complete Story',
       'Angelica Weaver: Catch Me When You Can', ..., 'Droid Assault',
       'Worms Ultimate Mayhem - Customization Pack DLC',
       'Train Simulator: Doncaster Works Route Add-On'], dtype=object)

In [89]:
juegos(2018)

array(['Lost Summoner Kitty', 'Ironbound', 'Battle Royale Trainer',
       'SNOW - All Access Basic Pass', 'SNOW - All Access Pro Pass',
       'SNOW - All Access Legend Pass', 'Race',
       'Army of Tentacles: (Not) A Cthulhu Dating Sim: Black GOAT of the Woods Edition',
       'Beach Rules', 'Planetarium 2 - Zen Odyssey',
       'The first thrust of God', 'SNOW - Starter Pack',
       'The first thrust of God - All Aircrafts', 'SNOW - Lifetime Pack',
       'The Warrior Of Treasures', 'DinoBlaster',
       'Robotpencil Presents: Understanding 3D for Concept',
       'Lost Cities', 'Twisted Enhanced Edition', 'Vaporwave Simulator',
       'Tactics Forever', 'Underdone - Soundtrack', 'Bitcoin Clicker',
       'Darkness and a Crowd', 'Shotgun Legend', 'My Time At Portia',
       'Watch Me Jump', 'Epic Cards Battle 2-Dragons Rising(TCG)',
       'HORDE ATTACK', 'MagiCat Original Soundtrack', 'Dark Devotion',
       'INTERSTELLAR PRIME', 'Reality Incognita', 'Island Build Masters',
     

In [80]:
# Función para obtener los specs más repetidos
@app.get("/specs/")
def specs(anio: int):
    df_filtrado = df[df['release_date'] == anio]
    if df_filtrado.empty:
        return {"!": f"No hay datos disponibles para el año {anio}"}
    
    specs_count = {}
    for lista_specs in df_filtrado['specs']:
        if lista_specs is not None:
            for spec in lista_specs:
                specs_count[spec] = specs_count.get(spec, 0) + 1
    
    top_specs = dict(sorted(specs_count.items(), key=lambda item: item[1], reverse=True)[:5])
    return {"top_specs": list(top_specs.keys())}


In [83]:
specs(2019)

{'top_specs': ['Single-player',
  'Steam Achievements',
  'Full controller support',
  'Steam Cloud']}

In [58]:
# Función para obtener la cantidad de juegos lanzados en un año con early access
@app.get("/earlyacces/")
def earlyacces(anio: int):
    df_filtrado = df[df['release_date'] == anio]
    if df_filtrado.empty:
        return {"!": f"No hay datos disponibles para el año {anio}"}
    frecuencias=df_filtrado["early_access"].sum()
    
    return frecuencias

In [59]:
earlyacces(1999)

0

In [60]:
# Función para obtener el análisis de sentimiento

@app.get("/sentiment/")
def sentiment(anio: int):
    df_filtrado = df[df['release_date'] == anio]
    if df_filtrado.empty:
        return {"!": f"No hay datos disponibles para el año {anio}"}
    frecuencia=df_filtrado["sentiment"].value_counts().to_dict()
    return frecuencia


In [61]:
sentiment(2016)

{'sin_calificar': 1556,
 'Mixed': 938,
 'Very Positive': 766,
 'Positive': 695,
 'Mostly Positive': 562,
 '1 user reviews': 512,
 '2 user reviews': 364,
 '3 user reviews': 277,
 '4 user reviews': 219,
 'Mostly Negative': 215,
 '5 user reviews': 199,
 '6 user reviews': 168,
 '7 user reviews': 158,
 '8 user reviews': 118,
 '9 user reviews': 108,
 'Overwhelmingly Positive': 49,
 'Negative': 23,
 'Very Negative': 5,
 'Overwhelmingly Negative': 1}

In [62]:
# Función para obtener los juegos top 5 con mayor metascore
@app.get("/metascore/")
def metascore(anio: int):
    # Filtrar el DataFrame por el año proporcionado
    df_anio = df[df["release_date"]== anio]

    # Eliminar duplicados basados en la columna "title"
    juegos_unicos = df_anio.drop_duplicates(subset="title")

    # Ordenar los juegos únicos por Metascore de manera descendente
    juegos_ordenados = juegos_unicos.sort_values(by="metascore", ascending=False)

    # Tomar los 5 juegos top con los Metascores más altos
    top_juegos = juegos_ordenados.head(5)

    # Crear una lista de diccionarios con la información de los juegos top
    juegos_top = top_juegos[["title", "metascore"]].to_dict(orient="records")

    return juegos_top

In [63]:
metascore(2019)

[{'title': 'Raji: An Ancient Epic', 'metascore': nan},
 {'title': 'The Legendary Player - Make Your Reputation - OPEN BETA',
  'metascore': nan},
 {'title': 'The End of an Age: Fading Remnants', 'metascore': nan}]